In [1]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import integrate
from scipy import optimize
from scipy import interpolate
from scipy import special

# Parallelizing
import multiprocessing as mp
import numba
from numba import njit, prange
import joblib
from joblib import Parallel, delayed
#----------------------------------------------

from mpl_toolkits import mplot3d
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable 
%pylab inline 
%config InlineBackend.figure_format = 'svg' 
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['mathtext.fontset'] = 'cm'
#Assume installed from github using "git clone --recursive https://github.com/cmbant/CAMB.git"
#This file is then in the docs folders
#camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
#sys.path.insert(0,camb_path)
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

Populating the interactive namespace from numpy and matplotlib
Using CAMB 1.1.1 installed at /home/alba/anaconda2/envs/ipykernel_py3/lib/python3.8/site-packages/camb


In [2]:
# Load file with transfers
cmb_transfer_CAMB = pd.read_csv('./Files/cmb_transfer_TT_EE.txt', sep=" ", header=None)
cmb_transfer_CAMB.columns=['ell','k','dk','gT','gP']

FileNotFoundError: [Errno 2] File ./Files/cmb_transfer_TT_EE.txt does not exist: './Files/cmb_transfer_TT_EE.txt'

In [58]:
# Selectes the values of ell in the dataframe
ell_values_CAMB = cmb_transfer_CAMB['ell'].unique()

In [13]:
# Compute r_sampling according to Table 1 of Liguori et al. PRD, 76, 105016 (2007)
tau0 = 14142.0762
r = tau0 + 500.0 
r_count = 0.0
r_values = [r] #starting from r

while r >105.0:  
    r_count+=1.0
    if r_count <= 450.0:
        r_sample = 3.5 #sample densely during recombination
    elif r_count <= 485.0:
        r_sample = 105.0
    elif r_count <= 515.0:
        r_sample = 10.0 #sample densely during reionization
    else:
        r_sample = 105.0
    r-=r_sample
    r_values.append(r)

$$\alpha (\ell, r) = \frac{2}{\pi} \int \mathrm{d}k k^2[g_{T\ell}(k)j_\ell(kr)]$$

In [15]:
def compute_alpha(ell,
                  values_of_r,
                  transfer_function,
                  source):
    
    reduced_df = transfer_function[transfer_function['ell'] == ell]
    array_alpha_r = np.zeros(len(values_of_r),'float64')
        
    for index_r in range(len(values_of_r)):
        array_alpha_r[index_r] = 2.0/np.pi * integrate.trapz(reduced_df[source].values*
                                                      special.spherical_jn(int(ell),
                                                      reduced_df['k'].values*values_of_r[index_r])*
                                                      reduced_df['k'].values*reduced_df['k'].values,
                                                      x = reduced_df['k'].values)
        
    return array_alpha_r

In [34]:
alpha_function_temp = Parallel(n_jobs=n_cpu)(delayed(compute_alpha)(ell,r_values,cmb_transfer_CAMB,'gT') 
                                             for ell in ell_values_CAMB.astype(int))

CPU times: user 450 ms, sys: 1.42 s, total: 1.87 s
Wall time: 13.8 s


In [19]:
alpha_5000 = compute_alpha(5000,r_values,cmb_transfer_CAMB,'gT') 

In [20]:
alpha_5000_P = compute_alpha(5000,r_values,cmb_transfer_CAMB,'gP') 

In [14]:
alpha_function_pol = Parallel(n_jobs=n_cpu)(delayed(compute_alpha)(ell,r_values,cmb_transfer_CAMB,'gP') 
                                            for ell in ell_values_CAMB.astype(int))

CPU times: user 19.3 s, sys: 11.9 ms, total: 19.3 s
Wall time: 19.3 s


In [49]:
alpha_function_temp = np.loadtxt('alpha_function_lmax5000_T.txt')
alpha_function_pol = np.loadtxt('alpha_function_lmax5000_P.txt')

In [52]:
alpha_function_temp = np.r_[alpha_function_temp,np.full((1,602), alpha_5000)] 

In [54]:
alpha_function_pol = np.r_[alpha_function_pol,np.full((1,602), alpha_5000_P)] 

In [55]:
alpha_function_pol.shape

(4999, 602)

In [56]:
np.savetxt('./alphabetagammadelta_files/alpha_function_lmax5000_T.txt',alpha_function_temp)

In [57]:
np.savetxt('./alphabetagammadelta_files/alpha_function_lmax5000_P.txt',alpha_function_pol)